In [1]:
import pandas as pd 
import numpy as np 
from Bio import SeqIO
from src.tools import NCBIDatasets

%load_ext autoreload
%autoreload 2

In [2]:
# Changing my training dataset strategy. Instead going to use the sequences directly from NCBI. 
genome_metadata_df = pd.read_csv('../data/genome_metadata.csv', index_col=0)
genome_ids = genome_metadata_df[genome_metadata_df.phylum == 'Campylobacterota'].index.values 


In [6]:
ncbi = NCBIDatasets()
ncbi.run(genome_ids=genome_ids, metadata=False, include=['protein'], dirs={'protein':'../data/ncbi/proteins'})
ncbi.cleanup()

NCBIDatasets._get_genome: Downloading data for GCF_900197775.1.: 100%|██████████| 197/197 [00:00<00:00, 2492.14it/s]
